In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [15]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


In [16]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
DATA_PATH = os.path.join('MP_DATA')

actions = np.array(['a','b','c','d','e','f','g',
                    'h','i','j','k','l','m','n',
                    'o','p','q','r','s','t','u','v',
                    'v','w','x','y','z'])

#no of videos
no_sequnces = 30

#number of frames
sequnce_length = 30 


In [25]:
for action in actions:
    for sequence in range(no_sequnces):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [26]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequnces):
            for frame_num in range(sequnce_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)
                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    cv2.putText(image,"STARTING COLLECTION",(120,120),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA,
                                )
                    cv2.putText(image,"Collecting frames for {} Video Number {}".format(action,sequence),(120,120),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA,
                                )
                    cv2.waitKey(2000)

                else:
                    cv2.putText(
                                image,
                                "Collecting frames for {} Video Number {}".format(action,sequence),(120,120),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA,
                                )

                keypoints = extract_keypoints(results)

                npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))

                np.save(npy_path,keypoints)


                # Show to screen
                cv2.imshow('OpenCV Feed', image)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

                
    cap.release()
    cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [23]:
cap.release()
cv2.destroyAllWindows()

In [27]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [34]:
label_map = {label:num for num,label in enumerate(actions)}

In [35]:
label_map

{np.str_('a'): 0,
 np.str_('b'): 1,
 np.str_('c'): 2,
 np.str_('d'): 3,
 np.str_('e'): 4,
 np.str_('f'): 5,
 np.str_('g'): 6,
 np.str_('h'): 7,
 np.str_('i'): 8,
 np.str_('j'): 9,
 np.str_('k'): 10,
 np.str_('l'): 11,
 np.str_('m'): 12,
 np.str_('n'): 13,
 np.str_('o'): 14,
 np.str_('p'): 15,
 np.str_('q'): 16,
 np.str_('r'): 17,
 np.str_('s'): 18,
 np.str_('u'): 19,
 np.str_('v'): 21,
 np.str_('w'): 22,
 np.str_('x'): 23,
 np.str_('y'): 24,
 np.str_('z'): 25}

In [37]:
sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequnces):
        window = []
        for frame_num in range(sequnce_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [38]:
x = np.array(sequences)

In [39]:
y = to_categorical(labels).astype(int)

In [40]:
X_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard


In [43]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [52]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation="relu",input_shape=(30,1662)))
model.add(LSTM(256,return_sequences=True,activation="relu"))

model.add(LSTM(128,return_sequences=True,activation="relu"))
model.add(LSTM(64,return_sequences=False,activation="relu"))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))





c:\Users\cva\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [53]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['categorical_accuracy'])

In [54]:
model.fit(X_train,y_train,epochs=2000,callbacks=[tb_callback])

Epoch 1/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - categorical_accuracy: 0.0487 - loss: 4.2710
Epoch 2/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - categorical_accuracy: 0.0661 - loss: 3.2677
Epoch 3/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - categorical_accuracy: 0.0763 - loss: 3.2404
Epoch 4/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - categorical_accuracy: 0.0719 - loss: 3.2367
Epoch 5/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - categorical_accuracy: 0.0831 - loss: 3.2332
Epoch 6/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - categorical_accuracy: 0.0822 - loss: 3.2308
Epoch 7/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - categorical_accuracy: 0.0858 - loss: 3.2404
Epoch 8/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - categorical_accuracy: 0.0865 - loss: 3.2278
Epoch 9/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - categorical_accuracy: 0.0780 - loss: 3.2314
Epoch 10/2000
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - categorical_accuracy: 0.0782 - loss: 3.2304
Epo

KeyboardInterrupt: 